In [19]:
from utils import db
from utils import s3
import pandas as pd
import random
import datetime

In [2]:

'''
1.Create table :tran_fact
tran_id int, cust_id varchar(20),tran_ammount decimal(10,2), tran_type varchar(1), tran_country_cd varchar(3),tran_date date
tran_id : random id (ex : FX_123456,TD_224452  (Concat [FX/TD] with 6 digit number)
cust_id : cust_12345 (concat cust+ 4 digit number)
tran_ammount : $ ammount ( 3 dgit to 10 digit number)
tran_type : C/D (Random Value)
country_cd : ['USA','CAN','IND','AFG','CHN','JPN','KON','PAL']
tran_date : ['2022-01-01' to '2022-03-10' ]

Generate records for every day with the date range given [ each day generate 1000 to 10000 records]
have 7 days fro now in tester
'''


"\n1.Create table :tran_fact\ntran_id int, cust_id varchar(20),tran_ammount decimal(10,2), tran_type varchar(1), tran_country_cd varchar(3),tran_date date\ntran_id : random id (ex : FX_123456,TD_224452  (Concat [FX/TD] with 6 digit number)\ncust_id : cust_12345 (concat cust+ 4 digit number)\ntran_ammount : $ ammount ( 3 dgit to 10 digit number)\ntran_type : C/D (Random Value)\ncountry_cd : ['USA','CAN','IND','AFG','CHN','JPN','KON','PAL']\ntran_date : ['2022-01-01' to '2022-03-10' ]\n\nGenerate records for every day with the date range given [ each day generate 1000 to 10000 records]\nhave 7 days fro now in tester\n"

In [28]:
""" 
CREATE SCHEMA IF NOT EXISTS hadoop;
DROP TABLE hadoop.tran_fact;
CREATE TABLE IF NOT EXISTS hadoop.tran_fact(
    tran_id varchar(18) encode raw, 
    cust_id varchar(20) encode raw,
    tran_ammount decimal(10,2) encode AZ64, 
    tran_type varchar(1) encode raw, 
    tran_country_cd varchar(3) encode bytedict,
    tran_date date encode delta32k
);
"""

# def generate_dummy_data(start: datetime, num_days : int, num_records : int ):
    
start_date = datetime.datetime(2022, 1, 1)
num_days = 7
num_records = 5000


def generate_dummy_data(start: datetime, num_days : int, num_records : int, filepath = 'data/hadoop/'):
    tran_country_cds = ['USA','CAN','IND','AFG','CHN','JPN','KON','PAL']
    tran_types = ['C', 'D']
    id_prefixes = ['FX', 'TD']
    header = 'tran_id,cust_id,tran_ammount,tran_type,tran_country_cd,tran_date\n'
    files_created = []
    for i in range(num_days):
        tran_date = start_date + datetime.timedelta(days=i)
        filename = tran_date.strftime("tran_fact_%Y-%m-%d") + ".csv"
        f = open(filepath + filename, 'w')
        f.write(header)
        for j in range(num_records):
            tran_ammount = round(random.uniform(100.00, 99999999.99),2)
            tran_type = random.choice(tran_types)
            tran_country_cd = random.choice(tran_country_cds)
            tran_id = random.choice(id_prefixes) + '_' + tran_date.strftime("%Y%m%d") + '_' +  f"{j+1:06}"
            cust_id = 'cust_' + f"{random.randint(1, 9999):04}"
            f.write(tran_id + "," + cust_id + "," + str(tran_ammount) + ","
                + tran_type + "," + tran_country_cd + "," + tran_date.strftime("%Y-%m-%d") + '\n')
        files_created.append((filename, filepath, tran_date.strftime('%Y-%m-%d')))
        f.close()
    return files_created
        
files_created = generate_dummy_data(start_date, num_days, num_records)

In [29]:
''' 
2. Load each file into s3. [ each with folder as 2022-01-01,2022-01-02]
'''
for f in files_created:
    s3.upload_file(f[1] + f[0], 'quintrix-spearscjs', 'cards_ingest/tran_fact/daily/' + f[2] + '/' + f[0])

In [6]:
'''
3.Create table in Redshift.
'''
query = 'CREATE SCHEMA IF NOT EXISTS hadoop'
db.do_query(query, [])
db.do_query_file('sql/create/ct_hadoop_tran_fact.sql',[])

sql/create/ct_hadoop_tran_fact.sql done.


In [7]:
'''
4. Copy data to redshift  [append mode]
'''
db.copy('tran_fact', 'hadoop',  's3://quintrix-spearscjs/hadoop/data/daily/', db.redshift_iam, ',')

In [10]:
'''
1. Unload query sum(tran)amt by country cd to parquet file in s3.
'''
subquery = 'SELECT tran_country_cd, SUM(tran_ammount) FROM hadoop.tran_fact GROUP BY tran_country_cd;'
bucket = 's3://quintrix-spearscjs/hadoop/data/country/tran_ammt_sum/'
query = """ 
    UNLOAD (%s) -- query string
    TO %s   -- s3 bucket_name
    IAM_ROLE %s -- redshift iam role
    PARALLEL ON
    ALLOWOVERWRITE 
    FORMAT PARQUET
"""
db.do_query(query, [subquery, bucket, db.redshift_iam])